# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow_addons.metrics import RSquare

In [ ]:
data = pd.read_csv('../input/videogamesales/vgsales.csv', index_col='Rank')

In [ ]:
data

# Preprocessing

In [ ]:
columns_to_drop = ['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

data.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
data

In [ ]:
data.isnull().sum()

In [ ]:
data['Year'] = data['Year'].fillna(data['Year'].mean())

In [ ]:
data = data.dropna(axis=0)

In [ ]:
data.isnull().sum()

## Encoding

In [ ]:
data

In [ ]:
data['Platform'].unique()

In [ ]:
data['Genre'].unique()

In [ ]:
counts = data['Publisher'].value_counts()

data['Publisher'] = data['Publisher'].apply(lambda x: 'Small Publisher' if counts[x] < 50 else x)

In [ ]:
data

In [ ]:
onehot_columns = ['Platform', 'Genre', 'Publisher']

In [ ]:
def onehot_encode(data, columns):
    for column in columns:
        dummies = pd.get_dummies(data[column])
        data = pd.concat([data, dummies], axis=1)
        data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
data = onehot_encode(data, onehot_columns)

In [ ]:
data

## Scaling

In [ ]:
y = data['Global_Sales']
X = data.drop('Global_Sales', axis=1)

In [ ]:
X

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X.shape

# Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
inputs = tf.keras.Input(shape=(91,))
x = tf.keras.layers.Dense(128, activation='relu')(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(
    optimizer=optimizer,
    loss='mse'
)


batch_size = 64
epochs = 18

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0
)

# Results

In [ ]:
plt.figure(figsize=(14, 10))

epochs_range = range(1, epochs + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs_range, train_loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
np.argmin(val_loss)

In [ ]:
y_pred = np.squeeze(model.predict(X_test))

result = RSquare()
result.update_state(y_test, y_pred)

print("R^2 Score:", result.result())

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
history.history['val_loss']